In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [2]:
train_data = pd.read_csv(r"C:\Users\kd67152\Desktop\Datasets\train.csv") 
train_data = train_data.fillna(0)
train_data = train_data.drop(["id"],axis=1)
train_data.head() 

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,2011-08-20,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9,4,0,13,22,1,0,52,4,5850000
1,2011-08-23,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15,3,0,15,29,1,10,66,14,6000000
2,2011-08-27,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10,3,0,11,27,0,4,67,10,5700000
3,2011-09-01,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11,2,1,4,4,0,0,26,3,13100000
4,2011-09-05,77,77.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,319,108,17,135,236,2,91,195,14,16331452


In [3]:
# Extracting years
train_data["timestamp"] = train_data["timestamp"].apply(lambda x: x[:4]).astype('float')

In [4]:
object_cols = train_data.select_dtypes(include=[np.object]).columns
for col in object_cols:
    vals = train_data[col].unique()
    train_data[col].replace(vals,np.arange(len(vals)),inplace=True)

C:\Users\kd67152\AppData\Local\Temp\ipykernel_18344\2124611039.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  object_cols = train_data.select_dtypes(include=[np.object]).columns


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [6]:
X_train,X_test,y_train,y_test = train_test_split(train_data.drop('price_doc', axis=1),train_data["price_doc"],test_size=0.3)

In [16]:
sel = SelectFromModel(LogisticRegression())
sel.fit(X_train, y_train)

C:\Users\kd67152\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SelectFromModel(estimator=LogisticRegression())

In [18]:
selected_features = X_train.columns[(sel.get_support())]
selected_features

Index(['area_m', 'raion_popul', 'full_all', 'male_f', 'female_f', 'work_all',
       'work_male', 'work_female', 'ekder_all', '16_29_all', '16_29_male',
       '16_29_female', 'trc_sqm_500', 'office_sqm_1000', 'trc_sqm_1000',
       'office_sqm_1500', 'trc_sqm_1500', 'office_sqm_2000', 'trc_sqm_2000',
       'office_sqm_3000', 'trc_sqm_3000', 'office_sqm_5000', 'trc_sqm_5000'],
      dtype='object')

In [ ]:

importances = model.feature_importances_
indices = np.argsort(importances)

plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), selected_features[indices])
plt.xlabel('Relative Importance')

In [20]:
X = pd.DataFrame(train_data[selected_features])
y = train_data["price_doc"]

# Linear Regression

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

model = LinearRegression().fit(X_train,y_train)
predections = model.predict(X_test)
model.score(X_test,y_test)

0.1226823155814516

# KNN

In [22]:
from sklearn.neighbors import KNeighborsRegressor

In [23]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [24]:
knn.score(X_test,y_test)

0.22601908868889908

# RandomForestRegressor

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
rf = RandomForestRegressor(n_estimators=100,n_jobs=-1,max_depth=7)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.24084001878551786

# Ridge Regression

In [27]:
from sklearn.linear_model import Ridge

In [28]:
ridge = Ridge(alpha=0.1)
ridge.fit(X_train,y_train)

C:\Users\kd67152\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.09906e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


Ridge(alpha=0.1)

In [29]:
ridge.score(X_test,y_test)

0.12268224774472436

# Lasso Regression


In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lossse = Lasso(alpha=0.1)
lossse.fit(X_train,y_train)

In [ ]:
lossse.score(X_test,y_test)